# Getting Started with py-trelliscope

This notebook demonstrates the basic workflow for creating interactive trelliscope displays in Python.

## What is Trelliscope?

Trelliscope provides a scalable way to visualize and explore collections of plots. Each row in your DataFrame represents one panel (visualization) along with metadata (cognostics) that enable filtering, sorting, and labeling.

## Installation

```bash
pip install py-trelliscope  # When published
```

For now, install in development mode:
```bash


cd py-trelliscope2
pip install -e .

In [6]:
#!pip install -e .

import pandas as pd
import numpy as np
from pathlib import Path

from trelliscope import Display

## Example 1: Minimal Display

Let's create a simple display with just a few panels and metadata.

In [7]:
# Create sample data
df = pd.DataFrame({
    'panel_id': [f'panel_{i}' for i in range(1, 11)],
    'category': ['A', 'B', 'C'] * 3 + ['A'],
    'score': np.random.uniform(50, 100, 10),
    'rank': range(1, 11)
})

print("Sample data:")
df.head()

Sample data:


,panel_id,category,score,rank
0,panel_1,A,51.915137,1
1,panel_2,B,84.988421,2
2,panel_3,C,80.296439,3
3,panel_4,A,51.026909,4
4,panel_5,B,64.704638,5


In [8]:
# Create display
display = Display(
    df,
    name="simple_display",
    description="A simple example display"
)

print(display)

Display(name='simple_display', n_panels=10, n_columns=4, panel_column=not set)


### Configure the Display

1. **Set panel column**: Specify which column contains the panel data
2. **Infer metas**: Automatically detect metadata types from DataFrame columns
3. **Configure layout**: Set grid dimensions and arrangement

In [9]:
# Configure display using method chaining
display = (
    display
    .set_panel_column('panel_id')
    .infer_metas()
    .set_default_layout(ncol=3, nrow=2)
    .set_default_labels(['category', 'score'])
)

print("\nConfigured display:")
print(display)
print(f"\nMeta variables: {display.list_meta_variables()}")


Configured display:
Display(name='simple_display', n_panels=10, n_columns=4, panel_column=set)

Meta variables: ['category', 'panel_id', 'rank', 'score']


### Write Display to Disk

This creates a directory with:
- `displayInfo.json` - Display configuration and metadata
- `metadata.csv` - Cognostics (metadata) for each panel

In [10]:
# Write display
output_path = display.write(render_panels=False)
print(f"Display written to: {output_path}")

# List created files
print("\nCreated files:")
for file in output_path.iterdir():
    print(f"  - {file.name}")

Display written to: trelliscope_output/simple_display

Created files:
  - displayInfo.json
  - metadata.csv


## Example 2: Display with Explicit Meta Variables

Instead of inferring, you can explicitly define metadata types with custom parameters.

In [11]:
# Create data with multiple types
df2 = pd.DataFrame({
    'panel_id': [f'viz_{i}' for i in range(1, 21)],
    'region': np.random.choice(['North', 'South', 'East', 'West'], 20),
    'sales': np.random.uniform(1000, 50000, 20),
    'date': pd.date_range('2024-01-01', periods=20, freq='D'),
    'profit_margin': np.random.uniform(0.1, 0.4, 20)
})

df2.head()

,panel_id,region,sales,date,profit_margin
0,viz_1,North,22155.178314,2024-01-01,0.167062
1,viz_2,West,44546.163707,2024-01-02,0.345852
2,viz_3,North,17419.472505,2024-01-03,0.214031
3,viz_4,South,32177.234024,2024-01-04,0.118357
4,viz_5,West,5531.064013,2024-01-05,0.289937


In [12]:
# Create display with explicit metas
display2 = (
    Display(df2, name="sales_dashboard", description="Regional sales analysis")
    .set_panel_column('panel_id')
    .add_meta_def('region', 'factor', levels=['North', 'South', 'East', 'West'])
    .add_meta_def('sales', 'currency', code='USD', digits=2)
    .add_meta_def('date', 'date', format='%Y-%m-%d')
    .add_meta_def('profit_margin', 'number', digits=3)
    .set_default_layout(ncol=4, nrow=5)
    .set_panel_options(width=600, height=400)
)

print(display2)
print(f"\nMeta variables: {display2.list_meta_variables()}")

Display(name='sales_dashboard', n_panels=20, n_columns=5, panel_column=set)

Meta variables: ['date', 'profit_margin', 'region', 'sales']


In [13]:
# Inspect a specific meta variable
sales_meta = display2.get_meta_variable('sales')
print(f"Sales meta type: {sales_meta.type}")
print(f"Currency code: {sales_meta.code}")
print(f"Digits: {sales_meta.digits}")

Sales meta type: currency
Currency code: USD
Digits: 2


In [14]:
# Write display
output2 = display2.write(force=True, render_panels=False)  # force=True to overwrite if exists
print(f"Display written to: {output2}")

Display written to: trelliscope_output/sales_dashboard


## Example 3: Mixed Inference and Explicit Metas

You can combine automatic inference with manual overrides.

In [15]:
# Create dataset
df3 = pd.DataFrame({
    'id': range(1, 16),
    'group': np.random.choice(['Control', 'Treatment A', 'Treatment B'], 15),
    'measurement': np.random.normal(100, 15, 15),
    'timestamp': pd.date_range('2024-01-01 09:00', periods=15, freq='2h', tz='UTC'),
    'is_valid': np.random.choice([True, False], 15)
})

df3.head()

,id,group,measurement,timestamp,is_valid
0,1,Control,108.560101,2024-01-01 09:00:00+00:00,False
1,2,Control,118.197725,2024-01-01 11:00:00+00:00,False
2,3,Treatment A,85.613010,2024-01-01 13:00:00+00:00,True
3,4,Control,118.008019,2024-01-01 15:00:00+00:00,False
4,5,Treatment B,92.828792,2024-01-01 17:00:00+00:00,True


In [16]:
# Infer most metas, customize specific ones
display3 = (
    Display(df3, name="experiment_results")
    .set_panel_column('id')
    .infer_metas()  # Infer all
    .add_meta_def('measurement', 'number', digits=2, replace=True)  # Override
    .set_default_layout(ncol=3)
    .set_default_labels(['group', 'measurement', 'is_valid'])
)

# List all metas
print("All meta variables:")
for name in display3.list_meta_variables():
    meta = display3.get_meta_variable(name)
    print(f"  {name}: {meta.type}")

All meta variables:
  group: factor
  id: number
  is_valid: factor
  measurement: number
  timestamp: time


In [17]:
# Write display
output3 = display3.write(force=True, render_panels=False)
print(f"Display written to: {output3}")

Display written to: trelliscope_output/experiment_results


## Inspecting Display Output

Let's look at the generated files.

In [18]:
import json

# Read displayInfo.json
with open(output3 / 'displayInfo.json', 'r') as f:
    display_info = json.load(f)

print("Display Info Structure:")
print(f"  Name: {display_info['name']}")
print(f"  Description: {display_info['description']}")
print(f"  Number of metas: {len(display_info['metas'])}")
print(f"  Layout: {display_info['state']['layout']}")
print(f"  Labels: {display_info['state']['labels']}")

Display Info Structure:
  Name: experiment_results
  Description: 
  Number of metas: 5
  Layout: {'ncol': 3, 'nrow': None, 'page': 1, 'arrangement': 'row'}
  Labels: ['group', 'measurement', 'is_valid']


In [19]:
# Read metadata CSV
metadata = pd.read_csv(output3 / 'metadata.csv')
print("\nMetadata CSV:")
metadata.head()


Metadata CSV:


,group,measurement,timestamp,is_valid
0,Control,108.560101,2024-01-01 09:00:00+00:00,False
1,Control,118.197725,2024-01-01 11:00:00+00:00,False
2,Treatment A,85.613010,2024-01-01 13:00:00+00:00,True
3,Control,118.008019,2024-01-01 15:00:00+00:00,False
4,Treatment B,92.828792,2024-01-01 17:00:00+00:00,True


## Meta Variable Types

py-trelliscope supports multiple meta variable types:

| Type | Description | Example |
|------|-------------|----------|
| `factor` | Categorical with levels | `['A', 'B', 'C']` |
| `number` | Numeric continuous | `digits=2, log=False` |
| `currency` | Monetary values | `code='USD', digits=2` |
| `date` | Date without time | `format='%Y-%m-%d'` |
| `time` | Datetime with time | `timezone='UTC'` |
| `href` | Hyperlinks | `label_col='link_text'` |
| `graph` | Sparklines/viz | `direction='up'` |

## Summary

Key steps to create a trelliscope display:

1. **Prepare data**: DataFrame with panel column + metadata columns
2. **Create Display**: `Display(df, name="...")`
3. **Set panel column**: `.set_panel_column('col_name')`
4. **Define metadata**: 
   - Auto: `.infer_metas()`
   - Manual: `.add_meta_def('col', 'type', ...)`
5. **Configure**: Layout, labels, panel options
6. **Write**: `.write()` to create output

The generated `displayInfo.json` and `metadata.csv` can be consumed by the trelliscopejs viewer for interactive exploration.

## Next Steps

- **Panel Rendering**: Generate actual visualizations (matplotlib, plotly, etc.)
- **JavaScript Viewer**: View displays in interactive web interface
- **Advanced Features**: Filters, sorts, custom views
- **Large Datasets**: Handle 100k+ panels efficiently